In [211]:
import pandas as pd
from scipy.signal import butter, filtfilt
from scipy.signal import find_peaks
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split

def load_ecg_data_and_annotations(ecg_file_path, annotation_file_path):
    ecg_data = pd.read_csv(ecg_file_path)
    annotations = pd.read_csv(annotation_file_path)

    return ecg_data, annotations

def _butter_filter(sequence):
    fs = 360  # Sampling frequency
    nyquist = 0.5 * fs
    low = 0.4 / nyquist
    high = 45 / nyquist

    b, a = butter(N=3, Wn=[low, high], btype='band')
    return filtfilt(b, a, sequence)

def apply_filter(ecg_data):
    filtered_data = ecg_data.copy()
    for lead in ['MLII', 'V1']:
        # Check if the lead is in the DataFrame
        if lead in ecg_data.columns:
            filtered_data[lead] = _butter_filter(ecg_data[lead].values)

    return filtered_data

def detect_r_peaks(ecg_lead, distance=180):
    peaks, _ = find_peaks(ecg_lead, distance=distance)
    return peaks


def get_metrics(results, metrics_names, metric_key):
    for name, value in zip(metrics_names, results):
        if metric_key in name:
            return value
    return None


def build_and_train_model(train_x, train_y, dropout_rate, lstm_units, batch_size, learning_rate, reg_learning_rate, num_lstm_layers, class_weight_dict, val_x=[], val_y=[]):
    model = Sequential()
    model.add(LSTM(lstm_units, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2]),
                   kernel_regularizer=l2(reg_learning_rate), 
                   recurrent_regularizer=l2(reg_learning_rate)))
    
    for i in range(1, num_lstm_layers):
        model.add(LSTM(lstm_units, return_sequences=True if i < num_lstm_layers - 1 else False,
                       kernel_regularizer=l2(reg_learning_rate), 
                       recurrent_regularizer=l2(reg_learning_rate)))
        
    model.add(Dropout(dropout_rate))
    model.add(Dense(train_y.shape[1], activation='softmax', kernel_regularizer=l2(reg_learning_rate)))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])
    model.fit(train_x, train_y, validation_data=(val_x, val_y) if len(val_x) != 0 else None, epochs=30, batch_size=batch_size, verbose=1, class_weight=class_weight_dict)

    if len(val_x) != 0:
        results = model.evaluate(val_x, val_y, verbose=0)
    else:
        results = model.evaluate(train_x, train_y, verbose=0)
    metrics_names = model.metrics_names

    accuracy = results[metrics_names.index('accuracy')]
    precision = get_metrics(results, metrics_names, 'precision')
    recall = get_metrics(results, metrics_names, 'recall')
    
    return model, accuracy, precision, recall


In [213]:
ecg_data_209, annotations_209 = load_ecg_data_and_annotations('../data/209/209.csv', '../data/209/209annotations.csv')
filtered_ecg_data_209 = apply_filter(ecg_data_209)

In [214]:
window_size = 180 # 180
r_peaks = detect_r_peaks(filtered_ecg_data_209['MLII'].values, distance=108)
r_peaks[:10]

array([  34,  190,  317,  425,  537,  660,  769,  890,  999, 1114])

In [215]:
segment_data = []

for r_peak in r_peaks:
    start = max(0, r_peak - window_size // 2)
    end = min(len(filtered_ecg_data_209), r_peak + window_size // 2)

    relevant_annotations = annotations_209[(annotations_209['Sample #'] >= start) & (annotations_209['Sample #'] <= end)]
    relevant_annotations = relevant_annotations[relevant_annotations['Type'].isin(['N', 'A'])]

    if not relevant_annotations.empty:
        closest_annotation = relevant_annotations.iloc[(relevant_annotations['Sample #'] - r_peak).abs().argsort()[:1]]
        label = closest_annotation['Type'].values[0]
        segment_data.append({'Start': start, 'End': end, 'Label': label})

segments = pd.DataFrame(segment_data)

In [216]:
# Remove last Row since window size < 180
segments.drop(segments.tail(1).index,inplace=True)

In [219]:
segments_feature_1 = []
segments_feature_2 = []
segment_labels = []


for index, row in segments.iterrows():
    start_index = int(row['Start'])
    end_index = int(row['End'])
    label = row['Label']
    
    segment_mlII = filtered_ecg_data_209['MLII'][start_index:end_index+1].values
    segment_v1 = filtered_ecg_data_209['V1'][start_index:end_index+1].values
    
    segments_feature_1.append(segment_mlII)
    segments_feature_2.append(segment_v1)
    segment_labels.append(label)

combined_segments = [np.column_stack((mlII, v1)) for mlII, v1 in zip(segments_feature_1, segments_feature_2)]
combined_segments_array = np.array([np.array(segment) for segment in combined_segments], dtype=object)

label_mapping = {'N': 0, 'A': 1}
integer_labels = np.array([label_mapping[label] for label in segment_labels])
one_hot_labels = to_categorical(integer_labels)

print(f"Combined Segments Shape: {combined_segments_array.shape}")
print(f"One-Hot Labels Shape: {one_hot_labels.shape}")


Combined Segments Shape: (3002, 38, 4)
Combined Segments Shape: (3002, 38, 4)
One-Hot Labels Shape: (3002, 2)


In [220]:
# Split data into Train/Test
train_x, test_x, train_y, test_y = train_test_split(
    combined_segments_array, one_hot_labels, test_size=0.2, random_state=42, stratify=one_hot_labels, shuffle=True
)

# Class Distribution of Train Set
integer_labels_from_one_hot = np.argmax(train_y, axis=1)
class_counts = np.bincount(integer_labels_from_one_hot)
class_names = ['N', 'A']
for class_name, count in zip(class_names, class_counts):
    print(f"Class {class_name}: {count}")

Class N: 2095
Class A: 306


In [221]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Standardise Train Set
nsamples, ntimesteps, nfeatures = train_x.shape
train_x_reshaped = train_x.reshape((nsamples*ntimesteps, nfeatures))

scaler = StandardScaler()
scaler.fit(train_x_reshaped)

train_x_standardised = scaler.transform(train_x_reshaped)
train_x_standardised = train_x_standardised.reshape((nsamples, ntimesteps, nfeatures))

In [75]:
class_weights = [{0: 1, 1: 1}] # {0: 0.6666666666666666, 1: 2.0}, {0: 1, 1: 2}, {0: 1, 1: 4}, {0: 1, 1: 6}
hyperparameter_space = {
    'class_weight_dict': class_weights,
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'lstm_units': [32, 64],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.01, 0.001, 0.0001],
    'num_lstm_layers': [2, 3, 4],
    'reg_learning_rate': [0.1, 0.01, 0.001]
}

In [222]:
integer_labels_from_one_hot = np.argmax(train_y, axis=1)
classes = np.unique(integer_labels_from_one_hot)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=integer_labels_from_one_hot)
class_weight_dict = {class_label: weight for class_label, weight in zip(classes, class_weights)}

hyperparameter_space['class_weight_dict'] = [class_weight_dict]
class_weight_dict

{0: 0.5730310262529833, 1: 3.923202614379085}

In [25]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

y_labels = np.argmax(train_y, axis=1)

n_iterations = 1
best_score = 0
best_params = {}

for iteration in range(n_iterations):
    chosen_params = { param: np.random.choice(values) for param, values in hyperparameter_space.items() }
    print(f"Current Hyperparameters: {chosen_params}")

    accuracies = []
    precisions = []
    recalls = []
    for train_index, val_index in kf.split(train_x_standardised, y_labels):
        train_x_fold, val_x_fold = train_x_standardised[train_index], train_x_standardised[val_index]
        train_y_fold, val_y_fold = train_y[train_index], train_y[val_index]

        model, accuracy, precision, recall = build_and_train_model(train_x_fold, train_y_fold, **chosen_params, val_x=val_x_fold, val_y=val_y_fold) # With or wihtout weights?
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)

        print(f"Fold Scores: Acc - {accuracy} Pr - {precision} Re - {recall}")
    
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    print(f"Current Mean Scores: Acc - {avg_accuracy} Pr - {avg_precision} Re - {avg_recall} , Current Hyperparameters: {chosen_params}")

    # Update best params etc.
    if avg_accuracy > best_score:
        best_score = avg_accuracy
        best_params = chosen_params
        best_metrics = {
            'accuracy': avg_accuracy,
            'precision': avg_precision,
            'recall': avg_recall
        }
        print(f"New best score: {avg_accuracy:.4f} with params: {best_params} and metrics: {best_metrics}")

# Final best results
print(f"Best score: {best_score:.4f}")
print(f"Best params: {best_params}")
print(f"Best metrics: {best_metrics}")

# TODO:
# Try out balancing (weighted, or oversampling)
# Try out diff. window sizes


Current Hyperparameters: {'class_weight_dict': {0: 0.5505510301868711, 1: 5.4454976303317535}, 'dropout_rate': 0.3, 'lstm_units': 64, 'batch_size': 64, 'learning_rate': 0.001, 'num_lstm_layers': 4, 'reg_learning_rate': 0.001}
Epoch 1/15


KeyboardInterrupt: 

In [223]:
best_params = {'class_weight_dict': {0: 0.5730310262529833, 1: 3.923202614379085}, 'dropout_rate': 0.5, 'lstm_units': 64, 'batch_size': 32, 'learning_rate': 0.001, 'num_lstm_layers': 2, 'reg_learning_rate': 0.01}
print(f"Class Weights: {best_params['class_weight_dict']}")
print(f"Dropout Rate: {best_params['dropout_rate']}")
print(f"LSTM Layers: {best_params['num_lstm_layers']}")
print(f"LSTM Units: {best_params['lstm_units']}")
print(f"Batch Size: {best_params['batch_size']}")
print(f"Learning Rate: {best_params['learning_rate']}")
print(f"Regularizer Learning Rate: {best_params['reg_learning_rate']}")

# Train final model with the best parameters
model, accuracy, precision, recall = build_and_train_model(train_x_standardised, train_y, **best_params)

Class Weights: {0: 0.5730310262529833, 1: 3.923202614379085}
Dropout Rate: 0.5
LSTM Layers: 2
LSTM Units: 64
Batch Size: 32
Learning Rate: 0.001
Regularizer Learning Rate: 0.01
Epoch 1/30
76/76 [==============================] - 6s 20ms/step - loss: 1.9410 - accuracy: 0.5302 - precision_15: 0.5302 - recall_15: 0.5302
Epoch 2/30
76/76 [==============================] - 2s 21ms/step - loss: 0.9886 - accuracy: 0.6147 - precision_15: 0.6147 - recall_15: 0.6147
Epoch 3/30
76/76 [==============================] - 2s 20ms/step - loss: 0.7938 - accuracy: 0.5473 - precision_15: 0.5473 - recall_15: 0.5473
Epoch 4/30
76/76 [==============================] - 2s 20ms/step - loss: 0.7416 - accuracy: 0.5556 - precision_15: 0.5556 - recall_15: 0.5556
Epoch 5/30
76/76 [==============================] - 2s 20ms/step - loss: 0.7148 - accuracy: 0.5989 - precision_15: 0.5989 - recall_15: 0.5989
Epoch 6/30
76/76 [==============================] - 2s 20ms/step - loss: 0.7032 - accuracy: 0.5702 - precision_15

KeyboardInterrupt: 

In [228]:
# Standardise Test Set
test_nsamples, test_ntimesteps, test_nfeatures = test_x.shape
test_x_reshaped = test_x.reshape((test_nsamples * test_ntimesteps, test_nfeatures))
test_x_standardised = scaler.transform(test_x_reshaped)
test_x_standardised = test_x_standardised.reshape((test_nsamples, test_ntimesteps, test_nfeatures))

In [227]:
test = model.evaluate(test_x_standardised, test_y, verbose=1)

InvalidArgumentError: Graph execution error:

Detected at node while/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 725, in start

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.

In [199]:
# Evaluate on Test Set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_x_standardised, test_y, verbose=1)
test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1_score}")

InvalidArgumentError: Graph execution error:

Detected at node while/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 725, in start

  File "/Users/hemranakhtari/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.

In [186]:
from sklearn.metrics import classification_report

predictions = model.predict(test_x_standardised)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_y, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=['N', 'A'])
print(report)

ValueError: in user code:

    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.11/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro